# Probabilistic Programming

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import daft
from IPython.display import Image
import pystan
import seaborn as sns

In [ ]:
import warnings
warnings.simplefilter("ignore",category=FutureWarning)

In [ ]:
from scipy.optimize import minimize

## Domain specific languages (DSL)

A simplified computer language for working in a specific domain. Some examples of DSLs that you are already familiar with include

- regular expressions for working with text
- SQL for working with relational databases
- LaTeX for typesetting documents

Probabilistic programming languages are DSLs for dealing with models involving random variables and uncertainty. We will introduce the `Stan` probabilistic programming languages in this notebook.

### Stan and PyStan references

- [Paper describing Stan](http://www.stat.columbia.edu/~gelman/research/unpublished/stan-resubmit-JSS1293.pdf)
- [Stan documentation](http://mc-stan.org/users/documentation/index.html)
- [Stan examples](https://github.com/stan-dev/example-models/wiki)
- [PyStan docs](http://pystan.readthedocs.org/en/latest/)
- [PyStan GitHub page](https://github.com/stan-dev/pystan)

### Other packages for probabilistic programming

There several alternative packages for probabilistic programming in Python. You might like to explore them by recreating the PyStan examples shown in this notebooks using the following:

- [PyMC3](https://github.com/pymc-devs/pymc3)
- [Edward](http://edwardlib.org)
- [ZhuSuan](https://github.com/thu-ml/zhusuan)

## Examples

In  general, the problem is set up like this:
    
- We have some observed outcomes $y$ that we want to model
- The model is formulated as a probability distribution with some parameters $\theta$ to be estimated 
- We want to estimate the posterior distribution of the model parameters given the data
$$
P(\theta \mid y) = \frac{P(y \mid \theta) \, P(\theta)}{\int P(y \mid \theta^*) \, P(\theta^*) \, d\theta^*}
$$
- For formulating a specification using probabilistic programming, it is often useful to think of how we would simulated a draw from the model

### Coin bias

We toss a coin $n$ times, observed the number of times $y$ it comes up heads, and want to estimate the expected proportion of times $\theta$ that it comes up heads. An appropriate likelihood is the binomial, and it is convenient to use the $\beta$ distribution as the prior. In this case, the posterior is also a beta distribution, and there is a simple closed form formula: if the prior is $\beta(a, b)$ and we observe $y$ heads and $n-y$ tails in $n$ tosses, then the posterior is $\beta(a+y, a+(n-y)$. 

#### Graphical model

In [ ]:
pgm = daft.PGM(shape=[2.5, 3.0], origin=[0, -0.5])

pgm.add_node(daft.Node("alpha", r"$\alpha$", 0.5, 2, fixed=True))
pgm.add_node(daft.Node("beta", r"$\beta$", 1.5, 2, fixed=True))
pgm.add_node(daft.Node("p", r"$p$", 1, 1))
pgm.add_node(daft.Node("n", r"$n$", 2, 0, fixed=True))
pgm.add_node(daft.Node("y", r"$y$", 1, 0, observed=True))

pgm.add_edge("alpha", "p")
pgm.add_edge("beta", "p")
pgm.add_edge("n", "y")
pgm.add_edge("p", "y")

pgm.render()
plt.close()
pgm.figure.savefig("bias.png", dpi=300)
pass

In [ ]:
Image("bias.png", width=400)

#### Analytical solution

Illustrating what $y$, $\theta$, posterior, likelihood, prior, MLE and MAP refer to.

In [ ]:
n = 100
h = 61
p = h/n
rv = stats.binom(n, p)
mu = rv.mean()

a, b = 10, 10
prior = stats.beta(a, b)
post = stats.beta(h+a, n-h+b)
ci = post.interval(0.95)

thetas = np.linspace(0, 1, 200)
plt.plot(thetas, prior.pdf(thetas), label='Prior', c='blue')
plt.plot(thetas, post.pdf(thetas), label='Posterior', c='red')
plt.plot(thetas, n*stats.binom(n, thetas).pmf(h), label='Likelihood', c='green')
plt.axvline((h+a-1)/(n+a+b-2), c='red', linestyle='dashed', alpha=0.4, label='MAP')
plt.axvline(mu/n, c='green', linestyle='dashed', alpha=0.4, label='MLE')
plt.xlabel(r'$\theta$', fontsize=14)
plt.legend(loc='upper left')
plt.show()
pass

## Using `stan`

### Coin bias

#### Data

In [ ]:
data = {
    'n': 100,
    'y': 61,
}

#### Model

In [ ]:
code = """
Write your Stan code here.
"""

#### Compile the C++ model

In [ ]:
sm = pystan.StanModel(model_code=code)

In [ ]:
print(sm.model_cppcode)

#### MAP

In [ ]:
fit_map = sm.optimizing(data=data)

In [ ]:
fit_map.keys()

In [ ]:
fit_map.get('p')

In [ ]:
fit = sm.sampling(data=data, iter=1000, chains=4)

Summarizing the MCMC fit

In [ ]:
print(fit.stansummary())

### Interpreting `n_eff` and `Rhat`

#### Effective sample size

$$
\hat{n}_{eff} = \frac{mn}{1 + 2 \sum_{t=1}^T \hat{\rho}_t}
$$

where $m$ is the number of chains, $n$ the number of steps per chain, $T$ the time when the autocorrelation first becomes negative, and $\hat{\rho}_t$ the autocorrelation at lag $t$.

##### Gelman-Rubin $\widehat{R}$

$$
\widehat{R} = \frac{\widehat{V}}{W}
$$

where $W$ is the within-chain variance and $\widehat{V}$ is the posterior variance estimate for the pooled traces.  Values greater than one indicate that one or more chains have not yet converged.

Discrad burn-in steps for each chain. The idea is to see if the starting values of each chain come from the same distribution as the stationary state. 

- $W$ is the number of chains $m \times$ the variacne of each individual chain
- $B$ is the number of steps $n \times$ the variance of the chain means
- $\widehat{V}$ is the weigthed average $(1 - \frac{1}{n})W + \frac{1}{n}B$

The idea is that $\widehat{V}$ is an unbiased estimator of $W$ if the starting values of each chain come from the same distribution as the stationary state. Hence if $\widehat{R}$ differs significantly from 1, there is probably no convergence and we need more iterations. This is done for each parameter $\theta$.

#### $\widehat{R}$ is a measure of chain convergence

In [ ]:
ps = fit.extract(None, permuted=False)

In [ ]:
fit.model_pars

In [ ]:
ps.shape

In [ ]:
fig, axes = plt.subplots(2,2)
for i, ax in enumerate(axes.ravel()):
    ax.plot(ps[:, i, 0])
    ax.set_title('Chain %d' % (i+1))
plt.tight_layout()

#### Plotting

In [ ]:
fit.plot()
pass

#### Extracting parameters

In [ ]:
params = fit.extract()

In [ ]:
p = params['p']
plt.subplot(121)
plt.hist(p, 20, alpha=0.5)
plt.subplot(122)
plt.plot(p, alpha=0.5)
pass

### Coin toss as Bernoulli model

In [ ]:
%%file bernoulli_model.stan

Write your Stan code here.

In [ ]:
y = np.random.choice([0,1], 100, p=[0.6, 0.4])
data = {
    'N': len(y),
    'y': y
}

In [ ]:
sm = pystan.StanModel('bernoulli_model.stan')

In [ ]:
fit = sm.sampling(data=data, iter=1000, chains=4)

In [ ]:
fit

In [ ]:
fit.plot()
pass

#### MAP

In [ ]:
opt = sm.optimizing(data)

In [ ]:
opt

The MAP maximizes the log probability of the model.

In [ ]:
xi = np.linspace(0, 1, 100)
plt.plot(xi, [fit.log_prob(np.log(x) - np.log(1-x)) for x in xi])
pass

Stan automatically transforms variables so as to work with unconstrained optimization. Knowing this, we can try to replicate the optimization procedure.

In [ ]:
p0 = 0.1
x0 = np.log(p0) - np.log(1 - p0)
sol = minimize(fun=lambda x: -fit.log_prob(x), x0=x0)

In [ ]:
sol

In [ ]:
np.exp(sol.x)/(1 + np.exp(sol.x))

### Linear regression

Another simple example of a probabilistic model is linear regression

$$
y = ax + b + \epsilon
$$

with $\epsilon \sim N(0, \sigma^2)$.

We can think of the simulation model as sampling $y$ from the probability distribution 

$$
y \sim N(ax + b, \sigma^2)
$$

and the parameter $\theta = (a, b, \sigma)$ is to be estimated (as posterior probability, MLE or MAP). To complete the model, we need to specify prior distributions for $a$, $b$ and $\sigma$. For example, if the observations $y$ are standardized to have zero mean and unit standard distribution, we can use

$$
a \sim N(0, 10) \\
b \sim N(0, 10) \\
\sigma \sim \vert{N(0, 1)}
$$

To get a more robust fit that is less sensitive to outliers, we can use a student-T distribution for $y$

$$
y \sim t(ax + b, \sigma^2, \nu)
$$

with an extra parameter $\nu$ for the degrees of freedom for which we also need to specify a prior.

In [ ]:
# Instantiate the PGM.
pgm = daft.PGM(shape=[4.0, 3.0], origin=[-0.3, -0.7])

# Hierarchical parameters.
pgm.add_node(daft.Node("alpha", r"$\alpha$", 0.5, 2))
pgm.add_node(daft.Node("beta", r"$\beta$", 1.5, 2))
pgm.add_node(daft.Node("sigma", r"$\sigma$", 0, 0))

# Deterministic variable.
pgm.add_node(daft.Node("mu", r"$\mu_n$", 1, 1))

# Data.
pgm.add_node(daft.Node("x", r"$x_n$", 2, 1, observed=True))
pgm.add_node(daft.Node("y", r"$y_n$", 1, 0, observed=True))

# Add in the edges.
pgm.add_edge("alpha", "mu")
pgm.add_edge("beta", "mu")
pgm.add_edge("x", "mu")
pgm.add_edge("mu", "y")
pgm.add_edge("sigma", "y")

# And a plate.
pgm.add_plate(daft.Plate([0.5, -0.5, 2, 2], label=r"$n = 1, \cdots, N$",
    shift=-0.1, rect_params={'color': 'white'}))

# Render and save.
pgm.render()
plt.close()
pgm.figure.savefig("lm.png", dpi=300)

In [ ]:
Image(filename="lm.png", width=400)

### Linear model

In [ ]:
%%file linear.stan

Write your Stan code here.

In [ ]:
n = 11
_a = 6
_b = 2
x = np.linspace(0, 1, n)
y = _a + _b*x + np.random.randn(n)

data = {
    'N': n,
    'x': x,
    'y': y
}

#### Saving and reloading compiled models

Since Stan models take a while to compile, we will define two convenience functions to save and load them. For example, this will allow reuse of the mode in a different session or notebook without recompilation.

In [ ]:
import pickle

def save(filename, x):
    with open(filename, 'wb') as f:
        pickle.dump(x, f, protocol=pickle.HIGHEST_PROTOCOL)
        
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
model_name = 'linear'
filename = '%s.pkl' % model_name
if not os.path.exists(filename):
    sm = pystan.StanModel('%s.stan' % model_name)
    save(filename, sm)
else:
    sm = load(filename)

We can inspect the original model from the loaded compiled version.

In [ ]:
print(sm.model_code)

In [ ]:
fit = sm.sampling(data)

In [ ]:
fit

#### Re-using the model on a new data set

In [ ]:
n = 121
_a = 2
_b = 1
x = np.linspace(0, 1, n)
y = _a*x + _b + np.random.randn(n)

data = {
    'N': n,
    'x': x,
    'y': y
}

In [ ]:
fit2 = sm.sampling(data)

In [ ]:
print(fit2.stansummary(pars=['alpha', 'beta', 'sigma']))

### Hierarchical models

Gelman's book has an example where the dose of a drug may be affected to the number of rat deaths in an experiment.

| Dose (log g/ml) | # Rats | # Deaths |
|-----------------|--------|----------|
| -0.896          | 5      | 0        |
| -0.296          | 5      | 1        |
| -0.053          | 5      | 3        |
| 0.727           | 5      | 5        |

We will model the number of deaths as a random sample from a binomial distribution, where $n$ is the number of rats and $p$ the probability of a rat dying. We are given $n = 5$, but we believe that $p$ may be related to the drug dose $x$. As $x$ increases the number of rats dying seems to increase, and since $p$ is a probability, we use the following model:

$$
y \sim \text{Bin}(n, p) \\
\text{logit}(p) = \alpha + \beta x \\
\alpha \sim \mathcal{N}(0, 5) \\
\beta \sim \mathcal{N}(0, 10)
$$

where we set vague priors for $\alpha$ and $\beta$, the parameters for the logistic model.

**Exercise**: Create the plate diagram for this model using `daft`.

### Hierarchical model in Stan

In [ ]:
data = dict(
    N = 4,
    x = [-0.896, -0.296, -0.053, 0.727],
    y = [0, 1, 3, 5],
    n = [5, 5, 5, 5],
)

In [ ]:
%%file dose.stan

Write your Stan code here.

In [ ]:
model_name = 'dose'
filename = '%s.pkl' % model_name
if not os.path.exists(filename):
    sm = pystan.StanModel('%s.stan' % model_name)
    save(filename, sm)
else:
    sm = load(filename)

In [ ]:
fit = sm.sampling(data=data)
fit

In [ ]:
alpha, beta, *probs = fit.get_posterior_mean()
a = alpha.mean()
b = beta.mean()

#### Logistic function

$$
f(x) = \frac{e^z}{1 + e^z}
$$

In [ ]:
def logistic(a, b, x):
    """Logistic function."""
    return np.exp(a + b*x)/(1 + np.exp(a + b*x))

In [ ]:
xi =  np.linspace(min(data['x']), max(data['x']), 100)
plt.plot(xi, logistic(a, b, xi))
plt.scatter(data['x'], [y_/n_ for (y_, n_) in zip(data['y'], data['n'])], c='red')
pass

#### Sampling from prior

In [ ]:
%%file dose_prior.stan

Write your Stan code here.

In [ ]:
sm = pystan.StanModel('dose_prior.stan')

In [ ]:
fit_prior = sm.sampling(data=data)

In [ ]:
alpha, beta, *probs, lp = fit_prior.get_posterior_mean()
a = alpha.mean()
b = beta.mean()
p = [prob.mean() for prob in probs]

In [ ]:
p

In [ ]:
y = np.random.binomial(5, p)

In [ ]:
y

In [ ]:
xi =  np.linspace(min(data['x']), max(data['x']), 100)
plt.plot(xi, logistic(a, b, xi))
plt.scatter(data['x'], [y_/n_ for (y_, n_) in zip(y, data['n'])], c='red')
pass

#### Sampling from posterior

In [ ]:
%%file dose_post.stan

Write your Stan code here.

In [ ]:
sm = pystan.StanModel('dose_post.stan')

In [ ]:
fit_post = sm.sampling(data=data)

In [ ]:
yp = fit_post.extract('yp')['yp']

In [ ]:
yp.shape

In [ ]:
np.c_[data['x'], yp.T[:, :6]]